# Sentiment Analysis using Tweets (Ukraine and Russia conflict) on Oil Price

#### Twitter API Setup
[Setting up of Code and Cleaning up tweets](https://medium.com/swlh/using-twitter-api-for-tweets-sentiment-analysis-590c97ce52d8)

In [5]:
import matplotlib.pyplot as plt 
%matplotlib inline 
import re 
import pandas as pd 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
import csv 
import string 
import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from wordcloud import WordCloud,STOPWORDS 
from nltk.stem.porter import PorterStemmer

#### Establish Connection with the Project Dashboard (Twitter Developer API)

In [6]:
nltk.download('stopwords')
n_words = set(stopwords.words('english')) # stop words list 
porter = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bhuwa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
def TwitterClient(): 
    # Keys and tokens from Twitter Dev Console 
    # Keys will not be shared on git
    
    # attempt authentication 
    try:
        # create access token and secret 
        auth = OAuthHandler(consumer_key, consumer_secret) 
        # set access token and secret 
        auth.set_access_token(access_token, access_token_secret)
        api = tweepy.API(auth) 
    except: 
        print("Error: authentication failed")
        
    return api 
    
print(TwitterClient())
        

#### Preprocessing of Text

In [14]:
def clean_up_tweet(tweet):
    # cleaning the tweet from @ tags and links 
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) | (\w+:\/\/\S+)", "", tweet).split())
    # converting all letters to lower case and replacing '-' with spaces
    tweet = tweet.lower().replace('-', ' ')
    table = str.maketrans('', '', string.punctuation+string.digits)
    tweet = tweet.translate(table) 
    # tokenising words
    tokens = word_tokenize(tweet) 
    stemmed = [porter.stem(word) for word in tokens]
    words = [w for w in stemmed if not w in n_words]
    
    tweet = ' '.join(words)
    return tweet

#### Analysing Sentiment 

analysis = TextBlob(tweet) 
senti = analysis.sentiment.polarity

if senti < 0: 
    emotion = "NEG"
elif senti > 0: 
    emotion = "POS" 
else: 
    emotion = "NEU"

In [21]:
def get_tweets(query, count = 10): 
    tweets = [] 
    try: 
        fetched_tweets = api.search_tweets(q = query, count = count) 
        for tweet in fetched_tweets:
            tweet= clean(tweet.text)
            analysis = TextBlob(tweet)
            senti= analysis.sentiment.polarity
            # labeling the sentiment
            if senti<0 :
                emotion = "NEG"
            elif senti>0:
                emotion= "POS"
            else:
                emotion= "NEU"
            # appending all data
            tweets.append((tweet, senti, emotion))
        return tweets
    except tweepy.TweepyException as e: 
        print("Error : " + str(e))

In [22]:
api = TwitterClient() 
tweets = get_tweets(query = "Russia Oil", count = 1000)

df = pd.DataFrame(tweets, columns=['tweets', 'senti', 'emotion'])
df = df.drop_duplicates()

Error : 403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve
